# Amines for Carbon Capture: DMET based fragmentation stratergy

## Introduction
The objective is to solve accurately the electronic structure problem $H|\psi> = E|\psi>$. For small molecules this is possible on a classical computer. Yet, precise calculations of the electronic structure of large molecules typically come at a high computational expense. For instance, the cost of doing the entire CI calculation grows exponentially with system size. Consequently, it becomes crucial to use an effective technique for cutting the computing cost while keeping accuracy while performing electronic structure calculations when we focus on large-sized molecules, those vital for industrial problems. 

One such approach is to break down a molecular system into its component parts and their environments, for each part separately, depending on the problem. Then, a less precise method than the one that will be used to compute the electrical structure of a fragment is employed to calculate the environment surrounding it. The electronic structure problem for a specific fragment is then precisely solved, taking into account the environment's quantum mechanical effects. The  accurate calculation is incorporated into the quantum mechanical description, which is then updated i.e., solved iteratively.

![molecule](../images/co2-ch3nh2/ch3nh2-co2-frag.png)

## DMET

A molecule is broken down into fragments using the DMET approach, and each fragment is viewed as an open quantum system that is entangled with every other piece, collectively constituting the fragment's environment (or "bath"). According to this framework, a highly accurate quantum chemistry approach, such as the complete CI method or a coupled-cluster method, is used to solve the following Hamiltonian in order to determine the electronic structure of a given fragment.

$$
H_I = \sum_{rs}^{frag+bath}\left[ h_{rs} + \sum_{mn}[(rs|mn) - (rn|ms)]D_{mn}^{(mf)env}\right] a_r^{\dagger}a_s + \sum_{pqrs}^{frag+bath} (pq|rs) a_p^{\dagger}a_r^{\dagger}a_sa_q - \mu\sum_r^{frag}a_r^{\dagger}a_r
$$

The expression $\sum_{mn}[(rs|mn) - (rn|ms)]D_{mn}^{(mf)env}$ describes the quantum mechanical effects of the environment on the fragment, where $D_{mn}^{(mf)env}$  is the mean-field electronic density obtained by solving the Hartree–Fock equation. The Hamiltonian's one-particle term incorporates the environmental quantum mechanical effects. By adjusting $\mu$, the additional term $\mu\sum_r^{frag}a_r^{\dagger}a_r$ makes sure that the system's overall number of electrons and the amount of electrons in each of the pieces are equal.

In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import py3Dmol
from dmet.dmet_problem_decomposition import DMETProblemDecomposition
from dmet.meta_lowdin_localization import meta_lowdin_localization
from dmet.ccsd_solver import CCSDSolver
from pyscf import gto
from utils import rel_err
import pickle
import pandas as pd
import time

## Classical Simulation: CO2 @ CH3NH2

We perform DMET calculations on $CO_2 @ CH_3NH_2$ system. We consider two fragments of the system as shown above. Each fragment is solved using CCSD as electronic structure solver. We need to define an orbital localization technique for DMET. Here we use Meta-Lowdin localization. The iterations optimises the global chemical potential $\mu_{global}$ such that the electrons in the fragments are closer to the actual number of electrons of the system. The optmisation uses newton-raphson method.

In [3]:
with open('ch3nh2-co2-geometry-min-energy.pickle', 'rb') as handle:
    geometry = pickle.load(handle)

In [4]:
temp_g = geometry.split(';')
temp_g

['C 0.6147 1.0867 0.8571',
 'N 1.9021 1.7634 1.0319',
 'H 0.0879 1.5325 0.0061',
 'H 0.6641 -0.0036 0.6869',
 'H -0.0036 1.257 1.7453',
 'H 2.4036 1.3486 1.8155',
 'H 2.4869 1.6039 0.2133',
 'C 2.170233333333334 3.4314 1.3116333333333337',
 'O 2.267633755406905 3.7299153392002724 -0.5617755051573678',
 'O 2.0728329112597628 3.1328846607997276 3.185042171824037']

In [5]:
nitrogen = temp_g.pop(1)
temp_g.insert(6, nitrogen)
temp_g

['C 0.6147 1.0867 0.8571',
 'H 0.0879 1.5325 0.0061',
 'H 0.6641 -0.0036 0.6869',
 'H -0.0036 1.257 1.7453',
 'H 2.4036 1.3486 1.8155',
 'H 2.4869 1.6039 0.2133',
 'N 1.9021 1.7634 1.0319',
 'C 2.170233333333334 3.4314 1.3116333333333337',
 'O 2.267633755406905 3.7299153392002724 -0.5617755051573678',
 'O 2.0728329112597628 3.1328846607997276 3.185042171824037']

In [6]:
geometry = ';'.join(temp_g)
geometry

'C 0.6147 1.0867 0.8571;H 0.0879 1.5325 0.0061;H 0.6641 -0.0036 0.6869;H -0.0036 1.257 1.7453;H 2.4036 1.3486 1.8155;H 2.4869 1.6039 0.2133;N 1.9021 1.7634 1.0319;C 2.170233333333334 3.4314 1.3116333333333337;O 2.267633755406905 3.7299153392002724 -0.5617755051573678;O 2.0728329112597628 3.1328846607997276 3.185042171824037'

In [7]:
mol = gto.Mole() # Instantiate the molecule class in PySCF
mol.atom = geometry  # The coordinates of the atoms of the 10-hydrogen-atom ring are defined above
mol.basis = "sto-3g" # Use "minao" as the basis set
mol.charge = 0 # Assign the charge of the molecule
# mol.spin =  # Assign the spin of the molecule
mol.build() # Build the molecule object

In [8]:
dmet_solver = DMETProblemDecomposition()

dmet_solver.electronic_structure_solver = CCSDSolver()
dmet_solver.electron_localization_method = meta_lowdin_localization

energy = dmet_solver.simulate(mol, [4,6])

Active space:  range(0, 30)
Time for two-electron integrals(low_scf_twoint): 0.14597225189208984
Dmet active orbitals: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Dmet active orbitals: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Number of active orbitals: 30
Number of electrons: 40
orbital occupancy: [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.] 30
orbital occupancy: []
Number of active electrons: 40
Time for two-electron integrals(core_twoint): 0.17094135284423828
0 C 1s    
0 C 2s    
0 C 2px   
0 C 2py   
0 C 2pz   
1 H 1s    
2 H 1s    
3 H 1s    
4 H 1s    
5 H 1s    
6 N 1s    
6 N 2s    
6 N 2px   
6 N 2py   
6 N 2pz   
7 C 1s    
7 C 2s    
7 C 2px   
7 C 2py   
7 C 2pz   
8 O 1s    
8 O 2s    
8 O 2px   
8 O 2py   
8 O 2pz   
9 O 1s    
9 O 2s    
9 O 2px   
9 O 2py   
9 O 2pz   
0 C 1s    
0 C 2s    
0 C 2px   
0 C 2py   
0 C 2pz   
1 H 1s    
2 H 1s    
3 H 1s    
4 H 1s    
5 H 1s    
6 N 1s 

Time for two-electron integrals(two_int): 0.22549748420715332
Time for frag_twoint: 0.34695959091186523
		One Particle Hamiltonian Constructed (16, 16)
		Fock Matrix Constructed (16, 16)
		Two Particle Hamiltonian Constructed (16, 16, 16, 16)

		Guess Orbitals Constructed (16, 16)

Time for SCF RHF object:  0.0001919269561767578
Time for ao2mo.restore:  0.00010061264038085938
Time for SCF:  0.5636551380157471
		SCF Calculation Done -81.9605812602033
		Fock Matrix Constructed (16, 16)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7f02cd4c23a0>

Number of orbitals:  16
Frozen orbitals:  None
CCSD time:  22.952247142791748
		Electronic Structure Solver Done -82.09986983855698

Lambda time:  17.190260410308838
1-RDM time:  0.10560917854309082
2-RDM time:  0.39737439155578613
(16, 16) (16, 16, 16, 16)
		CC One RDM Constructed (16, 16)
		CC Two RDM Constructed (16, 16, 16, 16)

		Fragment Energy Computed -76.17839567484216
		Total Energy RDM Computed -82.09986983589911
		O

In [9]:
ev = 27.2114
actual_energy = -279.029888426681*ev

In [10]:
total_energy = energy
print(f'Actual total energy: {actual_energy:.12f} Ha\n')
print(f"Total energy: {total_energy:.12f} Ha")
print(f"Total energy in ev: {total_energy*ev:.12f} ev\n")

rel_error = rel_err(actual_energy, total_energy*ev)
print(f'Relative error: {rel_error:.12f}')

Actual total energy: -7592.793905933788 Ha

Total energy: -279.088799279473 Ha
Total energy in ev: -7594.396952713461 ev

Relative error: 0.000211127393


## Results

The relative error of `ground state energy obtained using DMET` with the `classical reference solution` is within $10^{-3}$ Hartree. The results of the DMET-CCSD and classical full molecule CCSD methods almost coincide.

## DMET using VQE as fragment solver

In the next notebook we apply DMET using VQE as fragment solver for $CO_2$ adsorption in amine groups